<a href="https://colab.research.google.com/github/mohammel-debug/hmm-pos-tagger/blob/main/Implement_an_HMM_PoS_tagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [201]:
# dataset 1: CHILDES
!wget -q https://raw.githubusercontent.com/UniversalDependencies/UD_English-CHILDES/master/en_childes-ud-train.conllu
!wget -q https://raw.githubusercontent.com/UniversalDependencies/UD_English-CHILDES/master/en_childes-ud-dev.conllu
!wget -q https://raw.githubusercontent.com/UniversalDependencies/UD_English-CHILDES/master/en_childes-ud-test.conllu


# dataset 2: Web Text (EWT)
# Download EWT dataset with exact filenames
!wget -O /content/en_ewt-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-train.conllu
!wget -O /content/en_ewt-ud-dev.conllu   https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-dev.conllu
!wget -O /content/en_ewt-ud-test.conllu  https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-test.conllu



--2025-11-22 13:14:26--  https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15029817 (14M) [text/plain]
Saving to: ‘/content/en_ewt-ud-train.conllu’

/content/en_ewt-ud- 100%[===================>]  14.33M  --.-KB/s    in 0.1s    

2025-11-22 13:14:26 (133 MB/s) - ‘/content/en_ewt-ud-train.conllu’ saved [15029817/15029817]

--2025-11-22 13:14:26--  https://raw.githubusercontent.com/UniversalDependencies/UD_English-EWT/master/en_ewt-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connect

In [202]:
data_train_childes = open("/content/en_childes-ud-train.conllu", "r", encoding="utf-8").read()
data_dev_childes = open("/content/en_childes-ud-dev.conllu", "r", encoding="utf-8").read()
data_test_childes = open("/content/en_childes-ud-test.conllu", "r", encoding="utf-8").read()

print("There are {} sentences in the training set.".format(len(data_train_childes)))
print("There are {} sentences in the testing set.".format(len(data_test_childes)))
print("There are {} sentences in the development set.".format(len(data_dev_childes)))

There are 19558483 sentences in the training set.
There are 5194505 sentences in the testing set.
There are 2159081 sentences in the development set.


In [203]:
data_train_ewt = open("/content/en_ewt-ud-train.conllu", "r", encoding="utf-8").read()
data_dev_ewt = open("/content/en_ewt-ud-dev.conllu", "r", encoding="utf-8").read()
data_test_ewt = open("/content/en_ewt-ud-test.conllu", "r", encoding="utf-8").read()

print("There are {} sentences in the training set.".format(len(data_train_ewt)))
print("There are {} sentences in the testing set.".format(len(data_test_ewt)))
print("There are {} sentences in the development set.".format(len(data_dev_ewt)))

There are 15028897 sentences in the training set.
There are 1899745 sentences in the testing set.
There are 1897963 sentences in the development set.


In [204]:
print(data_train_childes[:500])
print("----------------------------------")
#print(data_train_ewt[:500])

# sent_id = 0
# original_sent_id = 1754162
# s_24_sent_id = 14944
# child_name = Adam
# childes_toks = you won't cut your finger what
# corpus_name = Brown
# gold_annotation = False
# child_age = 41.95363354483665
# child_gender = male
# speaker_role = Investigator
# type = question
# text = You won 't cut your finger what?
1	You	you	PRON	PRP	_	4	nsubj	4:nsubj	_
2	won	will	AUX	VBD	_	4	aux	4:aux	_
3	't	not	PART	RB	_	4	advmod	4:advmod	_
4	cut	cut	VERB	VB	_	0	root	0:root	_
5	your	your	PRON	PRP$	_	6
----------------------------------


In [205]:
%cd /content/hmm-pos-tagger

/content/hmm-pos-tagger


In [206]:
# CHILDES dataset
with open("en_childes-ud-train.conllu", "w", encoding="utf-8") as f:
    f.write(data_train_childes)

with open("en_childes-ud-dev.conllu", "w", encoding="utf-8") as f:
    f.write(data_dev_childes)

with open("en_childes-ud-test.conllu", "w", encoding="utf-8") as f:
    f.write(data_test_childes)

# EWT dataset
with open("en_ewt-ud-train.conllu", "w", encoding="utf-8") as f:
    f.write(data_train_ewt)

with open("en_ewt-ud-dev.conllu", "w", encoding="utf-8") as f:
    f.write(data_dev_ewt)

with open("en_ewt-ud-test.conllu", "w", encoding="utf-8") as f:
    f.write(data_test_ewt)


In [ ]:
#!git clone https://github.com/mohammel-debug/hmm-pos-tagger.git
#%cd hmm-pos-tagger
#!mv en_childes-ud-train.conllu hmm-pos-tagger/
#!mv en_childes-ud-dev.conllu   hmm-pos-tagger/
#!mv en_childes-ud-test.conllu  hmm-pos-tagger/

In [ ]:
import nltk
!pip install conllu
nltk.download('universal_tagset')


In [ ]:
def parse_data(file1):

  sentences = []
  pos_tags = []

  current_sent_tokens = []
  current_sent_tags = []

  lines = file1.strip().split('\n')

  for line in lines:
      line = line.strip()

      # Skip comments/metadata and empty lines
      if line.startswith("#") or not line:
          # If we have accumulated tokens for a sentence, save them now
          if current_sent_tokens:
              sentences.append(current_sent_tokens)
              pos_tags.append(current_sent_tags)
              current_sent_tokens = []
              current_sent_tags = []
          continue

      parts = line.split('\t')

      # Safety check: ensure line has enough columns
      if len(parts) < 4:
          continue

      # Skip multi-word ranges (e.g., "1-2") because the individual tokens follow
      if '-' in parts[0]:
          continue

      # Append Token (Column 1) and UPOS (Column 3)
      # Note: If you want the specific XPOS tag (e.g., PRP, VBD), change index 3 to 4
      current_sent_tokens.append(parts[1])
      current_sent_tags.append(parts[3])

  # Flush the final sentence if the file didn't end with a newline
  if current_sent_tokens:
      sentences.append(current_sent_tokens)
      pos_tags.append(current_sent_tags)

  # --- Output Results ---
  print(f"Total Sentences found: {len(sentences)}\n")
  return sentences, pos_tags


In [ ]:
sentences_train, pos_tags_train = parse_data(data_train)
sentences_dev, pos_tags_dev = parse_data(data_dev)
sentences_test, pos_tags_test = parse_data(data_test)


In [ ]:
tags = set()
for word_l, tag_l in zip(sentences_train, pos_tags_train):
  for tag in tag_l:
    tags.add(tag)
# retrieve the total number of tags in the tagset and the set of tags
print(f"Total number of tags: {len(tags)}")
print(tags)


In [ ]:
tags = list()
for word, tag_l in zip(sentences_train, pos_tags_train):
  for tag in tag_l:
    tags.append(tag)
freq = nltk.FreqDist(tags)
freq.plot()

In [ ]:
tags = list()
for word, tag_l in zip(sentences_dev, pos_tags_dev):
  for tag in tag_l:
    tags.append(tag)
freq = nltk.FreqDist(tags)
freq.plot()

In [ ]:
tags = list()
for word, tag_l in zip(sentences_test, pos_tags_test):
  for tag in tag_l:
    tags.append(tag)
freq = nltk.FreqDist(tags)
freq.plot()

In [ ]:
#train_set = list()
# for word_l, tag_l in zip(sentences_test, pos_tags_test):
#   for word, tag in zip(word_l, tag_l):
# #     train_set.append((word,tag))
# train_set = list(zip(zip(sentences_train, pos_tags_train)))
# print(train)
tagged_sentences_train = [list(zip(s, p)) for s, p in zip(sentences_train, pos_tags_train)]
tagged_sentences_test = [list(zip(s, p)) for s, p in zip(sentences_test, pos_tags_test)]
tagged_sentences_dev = [list(zip(s, p)) for s, p in zip(sentences_dev, pos_tags_dev)]

In [ ]:
tagger = nltk.tag.hmm.HiddenMarkovModelTagger.train(tagged_sentences_train)

In [ ]:
tagger.accuracy(tagged_sentences_test)

TODO

- Same method. Try with the other dataset
- Combine datasets and train.
- Test with one and the other dataset. Combined.
- Plot the accuracies. Analyse.
- How to improve.
- Analyse the error per tag, and the reason. For each model.
- Check nltk for interesting methods.
- See if we can add more things from the unfinished chunking lab.